In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = "/Users/AnaPSilva/Documents/Ana/Ironhack/Bootcamp/Final_Project/Data/Rap/kanye.txt"

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 330453 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

"Kanye, can I talk to you for a minute? Me and the other faculty members was wonderin' could you do a lil som... Somethin' beautiful, somethin' that the kids is gon' love when they hear it. Tha's gon make them start jumpin' up and down and sharin' ca


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

84 unique characters


In [6]:
## Before training, you need to convert the strings to a numerical representation.
## convert each character into a numeric ID. 
## It just needs the text to be split into tokens first.
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

2022-03-12 16:19:20.094297: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [7]:
## from tokens to character IDs
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[59, 60, 61, 62, 63, 64, 65], [82, 83, 84]]>

In [8]:
## it will also be important to invert this representation and 
## recover human-readable strings from it.
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [9]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [10]:
## join the characters back into strings.
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [11]:
def text_from_ids(ids):
  """join the characters back into strings"""
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

#### Create training examples and targets

In [12]:
## convert the text vector into a stream of character indices.
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(330453,), dtype=int64, numpy=array([ 5, 41, 59, ...,  5,  2,  1])>

In [13]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

"
K
a
n
y
e
,
 
c
a


In [14]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
examples_per_epoch

3271

In [15]:
## The batch method lets you easily convert 
## these individual characters to sequences of the desired size.
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'"' b'K' b'a' b'n' b'y' b'e' b',' b' ' b'c' b'a' b'n' b' ' b'I' b' '
 b't' b'a' b'l' b'k' b' ' b't' b'o' b' ' b'y' b'o' b'u' b' ' b'f' b'o'
 b'r' b' ' b'a' b' ' b'm' b'i' b'n' b'u' b't' b'e' b'?' b' ' b'M' b'e'
 b' ' b'a' b'n' b'd' b' ' b't' b'h' b'e' b' ' b'o' b't' b'h' b'e' b'r'
 b' ' b'f' b'a' b'c' b'u' b'l' b't' b'y' b' ' b'm' b'e' b'm' b'b' b'e'
 b'r' b's' b' ' b'w' b'a' b's' b' ' b'w' b'o' b'n' b'd' b'e' b'r' b'i'
 b'n' b"'" b' ' b'c' b'o' b'u' b'l' b'd' b' ' b'y' b'o' b'u' b' ' b'd'
 b'o' b' ' b'a'], shape=(101,), dtype=string)


In [16]:
## It's easier to see what this is doing if you join the 
## tokens back into strings

for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'"Kanye, can I talk to you for a minute? Me and the other faculty members was wonderin\' could you do a'
b" lil som... Somethin' beautiful, somethin' that the kids is gon' love when they hear it. Tha's gon ma"
b"ke them start jumpin' up and down and sharin' candy an' stuff. Think you could probably do somethin' "
b'for the kids for graduation to sing?","[Intro]\r\nOh yeah, I\'ve got the perfect song for the kids to si'
b"ng\r\nAnd all my people that's...\r\n\r\n[Hook]\r\nDrug dealin' just to get by\r\nStack ya' money 'til it get s"


In [17]:
def split_input_target(sequence):
    """takes a sequence as input, duplicates, 
    and shifts it to align the input and label for each timestep"""
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [18]:
dataset = sequences.map(split_input_target)
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'"Kanye, can I talk to you for a minute? Me and the other faculty members was wonderin\' could you do '
Target: b"Kanye, can I talk to you for a minute? Me and the other faculty members was wonderin' could you do a"


#### Create training batches

In [19]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

#### Build The Model

- **tf.keras.layers.Embedding:** The input layer. A trainable lookup table that will map each character-ID to a vector with embedding_dim dimensions;
- **tf.keras.layers.GRU:** A type of RNN with size units=rnn_units (You can also use an LSTM layer here.)
- **tf.keras.layers.Dense:** The output layer, with vocab_size outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [20]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
display(vocab_size)
# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

84

In [21]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [22]:
model = MyModel(
    # Making sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

#### Try the model

In [23]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 85) # (batch_size, sequence_length, vocab_size)


In [24]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  21760     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  87125     
                                                                 
Total params: 4,047,189
Trainable params: 4,047,189
Non-trainable params: 0
_________________________________________________________________


In [25]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

## This gives us, at each timestep, a prediction of the next character index
sampled_indices

array([79,  5, 34, 10, 40, 46, 59, 11, 80, 63, 42, 15, 11, 78, 61,  8, 69,
       56, 52, 68, 61, 46, 36, 20,  9, 19, 41, 63, 81,  5,  5, 12, 35, 59,
       60, 41, 63, 38, 24, 20, 55, 44, 56, 49, 71, 36, 71, 71, 61, 44, 67,
       16,  4, 71,  3, 29,  9, 75, 75,  5, 61, 19, 81,  5, 63, 40, 71, 64,
       74, 18, 65, 26, 12, 54, 68, 59,  6, 16, 78, 75, 27, 12, 54, 54, 71,
       72, 83, 22, 77, 76, 84, 76,  2, 46, 78, 52, 45, 62, 83, 41])

In [26]:
## Decode these to see the text predicted by this untrained model
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"ll self-conscious, I'm just the first to admit it\r\n\r\n[Hook: Syleena Johnson + Kanye West]\r\nOh, when "

Next Char Predictions:
 b'u"D(JPa)veL-)tc&kZVjcPF2\'1Kew""*EabKeH62YNZSmFmmcNi.!m ;\'qq"c1w"eJmfp0g8*Xja#.tq9*XXmny4srzr\rPtVOdyK'


#### Train the model
- At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

In [27]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [28]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 85)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.4419947, shape=(), dtype=float32)


In [29]:
tf.exp(example_batch_mean_loss).numpy()

84.94421

In [30]:
## training procedure using the tf.keras.Model.compile method. 
## Use tf.keras.optimizers.Adam with default arguments and the loss function.
model.compile(optimizer='adam', loss=loss)

In [31]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_rap_kanye_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [33]:
EPOCHS = 200
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/200
51/51 [==============================] - 199s 4s/step - loss: 3.2718
Epoch 2/200
51/51 [==============================] - 213s 4s/step - loss: 2.4616
Epoch 3/200
51/51 [==============================] - 183s 4s/step - loss: 2.2566
Epoch 4/200
51/51 [==============================] - 206s 4s/step - loss: 2.1058
Epoch 5/200
51/51 [==============================] - 195s 4s/step - loss: 1.9629
Epoch 6/200
51/51 [==============================] - 149s 3s/step - loss: 1.8406
Epoch 7/200
51/51 [==============================] - 157s 3s/step - loss: 1.7394
Epoch 8/200
51/51 [==============================] - 155s 3s/step - loss: 1.6496
Epoch 9/200
51/51 [==============================] - 171s 3s/step - loss: 1.5668
Epoch 10/200
51/51 [==============================] - 155s 3s/step - loss: 1.4900
Epoch 11/200
51/51 [==============================] - 173s 3s/step - loss: 1.4208
Epoch 12/200
51/51 [==============================] - 147s 3s/step - loss: 1.3504
Epoch 13/200
51/51 [=====

In [34]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [35]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [38]:
start = time.time()
states = None
next_char = tf.constant(['Yo '])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Yo anybond them Does piece of)
My favorite about the dopem death
Five slad do stain up enough to get the fake of them TVacks
My mill awards age boya in a will

[Verse 2: Kanye West]
When the sun go down it's the magic time the game get
And if you slipped off the side, you don't know my styphe gost die Morkin
That's all it world let 'em heg
And now we gone for twenty years doing time behind bars
And since I gone to another not
I need every inderda repust is what I can't be bitched up on a live in Bash is coming and brought it to done
On the daught you should help shit on my grind
You do you and I'm just gon' do mine
Y'all niggas can't fuck with 'Ye
Y'all niggas can't fuck with 'Ye
Y'all niggas can't fuck with 'fore I'm talkin' to you!

[Verse 2: Kanye West]
One day I gues, they my money robe they
Don't have to take my folk soul
Tnow this video house over in the Maybacty ol where to go (now)

[Joke Feelia, Liearn Frong & Kanye West]
Cloades again
 aby, I got a plan

_____________________

In [37]:
tf.saved_model.save(one_step_model, 'Kanye West')

2022-03-13 02:09:48.933892: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: Kanye West/assets


INFO:tensorflow:Assets written to: Kanye West/assets
